# Cytokine prediction

In this section, we try to predict the cytokines for an unseen TCB

In [2]:
from __future__ import print_function, division
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import glob
from scifAI.ml import AutoFeatureSelection
from scifAI.utils import list_of_dict_to_dict
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.utils import shuffle
import skimage
plt.ion()   # interactive mode
from skimage.transform import   rescale, resize 
#from imageio import imread
import random
#import umap
from tqdm import tqdm
from random import sample
from skimage.transform import resize
import h5py
from skimage.exposure import rescale_intensity
import seaborn as sns
from umap import UMAP

In [3]:
from skimage.util import crop

def crop_pad_h_w(image_dummy,reshape_size):
    if image_dummy.shape[0] < reshape_size:
        h1_pad = ( reshape_size - image_dummy.shape[0])/2
        h1_pad = int(h1_pad)
        h2_pad =  reshape_size - h1_pad - image_dummy.shape[0]
        h1_crop = 0
        h2_crop = 0
    else:
        h1_pad = 0
        h2_pad = 0
        h1_crop = ( reshape_size - image_dummy.shape[0])/2
        h1_crop = abs(int(h1_crop))
        h2_crop = image_dummy.shape[0]- reshape_size  - h1_crop

    if image_dummy.shape[1] < reshape_size:
        w1_pad = (reshape_size - image_dummy.shape[1])/2
        w1_pad = int(w1_pad)
        w2_pad = reshape_size - w1_pad - image_dummy.shape[1]
        w1_crop = 0
        w2_crop = 0
    else:
        w1_pad = 0
        w2_pad = 0
        w1_crop = (reshape_size - image_dummy.shape[1])/2
        w1_crop = abs(int(w1_crop))
        w2_crop = image_dummy.shape[1]- reshape_size  - w1_crop

    h = [h1_crop, h2_crop, h1_pad, h2_pad]
    w = [w1_crop, w2_crop, w1_pad, w2_pad] 
    return h, w

In [4]:
color_map = np.array([
 
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
 (0.8352941176470589, 0.3686274509803922, 0.0),
    (0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8, 0.47058823529411764, 0.7372549019607844),
    (0.792156862745098, 0.5686274509803921, 0.3803921568627451),
 (0.9254901960784314, 0.8823529411764706, 0.2)])

lets load the data

In [5]:
metadata = pd.read_csv("/projects/site/pred/data_science/year/2022/immunological_synapse/synapse_data_features/metadata_subset_with_predictions.csv.gz")
metadata

,file,experiment,donor,condition,object_number,set,label,prediction,entropy,key,outlier,cluster
0,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_1,Donor_1,+SEA,5156,unlabeled,-1,B_cell,0.001350,Experiment_1 Donor_1,False,0
1,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_1,Donor_1,+SEA,32020,unlabeled,-1,T_cell,0.000959,Experiment_1 Donor_1,False,1
2,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_1,Donor_1,+SEA,55974,unlabeled,-1,Dead_Cell,0.002358,Experiment_1 Donor_1,False,0
3,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_1,Donor_1,+SEA,15150,unlabeled,-1,T_cell,0.000502,Experiment_1 Donor_1,False,1
4,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_1,Donor_1,+SEA,44814,unlabeled,-1,B_cell_with_small_area,0.028317,Experiment_1 Donor_1,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1065962,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_4,Donor_9,DIG-TCB,72493,unlabeled,-1,T_cell_with_signaling,0.259178,Experiment_4 Donor_9,False,1
1065963,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_4,Donor_9,DIG-TCB,106656,unlabeled,-1,B_cell,0.006382,Experiment_4 Donor_9,False,0
1065964,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_4,Donor_9,DIG-TCB,110990,unlabeled,-1,T_cell_with_signaling,0.487638,Experiment_4 Donor_9,False,1
1065965,/pstore/data/DS4/synapse_formation_full_data/E...,Experiment_4,Donor_9,DIG-TCB,71451,unlabeled,-1,B_cell,0.082181,Experiment_4 Donor_9,False,2


In [11]:
set_of_interesting_classes = ['B_cell', 
                              'T_cell', 
                              'T_cell_with_signaling',
                              'B_T_cell_in_one_layer',
                              'T_cell_with_B_cell_fragments', 
                              'Synapses_without_signaling',
                              'Synapses_with_signaling',
                              'No_cell_cell_interaction',
                              'Multiplets', ] 

In [12]:
row_index = metadata.label.isin(set_of_interesting_classes)
metadata.loc[row_index,:].groupby(["condition"]).count()

,file,experiment,donor,object_number,set,label,prediction,entropy,key,outlier,cluster
condition,,,,,,,,,,,
+SEA,4061,4061,4061,4061,4061,4061,4061,4061,4061,4061,4061
-SEA,1160,1160,1160,1160,1160,1160,1160,1160,1160,1160,1160
CD19-TCB,399,399,399,399,399,399,399,399,399,399,399
Teplizumab,228,228,228,228,228,228,228,228,228,228,228


5221

In [ ]:

df_features = pd.read_csv("/projects/site/pred/data_science/year/2022/immunological_synapse/synapse_data_features/df_features_subset.csv.gz")

In [ ]:
metadata

In [ ]:
metadata.outlier.unique()

In [ ]:
metadata.condition.unique()

In [ ]:
metadata.donor.unique() 

we need to load the frequency data which was calculated previously

In [ ]:

image_size = 128
channels_to_be_plotted = [0,3,4,5,6]
channel_colors = ["Greys","Oranges" ,"Blues","Greens","Reds"]

files = []
k = 5

indx = metadata.prediction == 'Synapses_with_signaling'
indx = indx & (metadata.outlier.astype(str) == 'False')
indx = indx & (metadata.condition.astype(str) == 'CD19-TCB')
indx = indx & (metadata.experiment.astype(str) == 'Experiment_4')
indx = indx & (metadata.donor.astype(str) == 'Donor_7')
indx = indx & (metadata.entropy < 0.7)
indx = indx[indx].sample(k,replace=False, random_state=0).index
cd19_files = metadata.loc[indx, "file"].sample(k,replace=False, random_state=0)
cd19_files = cd19_files.str.replace("/pstore/data/DS4/","/projects/site/pred/data_science/year/2022/immunological_synapse/").tolist()


indx = metadata.prediction == 'Synapses_with_signaling'
indx = indx & (metadata.outlier.astype(str) == 'False')
indx = indx & (metadata.condition.astype(str) == 'CD20-TCB')
indx = indx & (metadata.experiment.astype(str) == 'Experiment_4')
indx = indx & (metadata.donor.astype(str) == 'Donor_7')
indx = indx & (metadata.entropy < 0.7)
indx = indx[indx].sample(k,replace=False, random_state=0).index
cd20_files = metadata.loc[indx, "file"].sample(k,replace=False, random_state=0)
cd20_files = cd20_files.str.replace("/pstore/data/DS4/","/projects/site/pred/data_science/year/2022/immunological_synapse/").tolist()


files = cd19_files + cd20_files
nrows = len(files)
ncols = len(channels_to_be_plotted)

fix,ax = plt.subplots(nrows, ncols, figsize = ( ncols*1, nrows*1 ))
    
vmin = [4095.]*ncols
vmax = [0.]*ncols

for i in range(nrows):
    h5_file_ = h5py.File(files[i], "r")
    image = h5_file_.get("image")[()]
    mask = h5_file_.get("mask")[()]
    for j, ch in enumerate(channels_to_be_plotted):
        if j>0:
            vmin[j] = min(vmin[j], np.quantile(image[:,:,ch], 0.60)  )
            vmax[j] = max(vmax[j], np.quantile(image[:,:,ch], 0.99)  )
        else:
            vmin[j] = min(vmin[j], np.quantile(image[:,:,ch], 0.10)  )
            vmax[j] = max(vmax[j], np.quantile(image[:,:,ch], 0.99)  )

        
print(vmin,vmax)
for i in range(nrows):
    h5_file_ = h5py.File(files[i], "r")
    image = h5_file_.get("image")[()]
    h, w = crop_pad_h_w(image, image_size)
    h1_crop, h2_crop, h1_pad, h2_pad =  h 
    w1_crop, w2_crop, w1_pad, w2_pad = w 
    for j, ch in enumerate(channels_to_be_plotted):
            image_dummy = crop(image[:,:,ch], ((h1_crop,h2_crop),(w1_crop,w2_crop)))
            image_dummy = np.pad(image_dummy, 
                                     ((h1_pad,h2_pad),(w1_pad,w2_pad)), 
                                     'constant', 
                                     constant_values=np.median(image_dummy) )
            if j == 0:
                image_dummy[int(0.95*image_size):int(0.95*image_size+2), 
                            int(0.1*image_size):int(0.1*image_size  +25)] = vmax[j]
            ax[i,j].imshow(image_dummy, cmap = channel_colors[j] , vmin = vmin[j], vmax = vmax[j]  )
            ax[i,j].axis('off')
    h5_file_.close()

plt.savefig('Supplementary Figure - CD19 vs CD20 samples Donor 7.svg', bbox_inches="tight")

In [ ]:
indx = metadata.prediction == 'Synapses_with_signaling'
indx = indx & (metadata.outlier.astype(str) == 'False')
indx = indx & (metadata.condition.astype(str) == 'CD19-TCB')
indx = indx & (metadata.experiment.astype(str) == 'Experiment_4')
indx = indx & (metadata.donor.astype(str) == 'Donor_2')
indx = indx & (metadata.entropy < 0.7)
indx = indx[indx].sample(k,replace=False, random_state=0).index

cols = [ "std_intensity_MHCII", "mask_based_eccentricity_F-Actin"]

df_features.loc[indx, cols].quantile(0.95)

In [ ]:
indx = metadata.prediction == 'Synapses_with_signaling'
indx = indx & (metadata.outlier.astype(str) == 'False')
indx = indx & (metadata.condition.astype(str) == 'CD20-TCB')
indx = indx & (metadata.experiment.astype(str) == 'Experiment_4')
indx = indx & (metadata.donor.astype(str) == 'Donor_2')
indx = indx & (metadata.entropy < 0.7)
indx = indx[indx].sample(k,replace=False, random_state=0).index

cols = [ "std_intensity_MHCII", "mask_based_eccentricity_F-Actin"]

df_features.loc[indx, cols].quantile(0.95)